In [1]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = 'drive/MyDrive/imitate-retrieve-paraphrase'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install datasets
%pip install transformers
%pip install accelerate -U

In [3]:
from datasets import Dataset, load_dataset, DatasetDict, load_from_disk
import nltk
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq
import random
import torch
import tqdm

In [4]:
ds = load_from_disk(drive_path + '/data')

In [5]:
train_data = ds['train']

In [6]:
max_input = 512
max_target = 128

def preprocess_function(examples):
    inputs = []

    for i in range(len(examples['title'])):
        topic = examples['title'][i]
        style = examples['style'][i]
        fact = ''.join(random.sample(examples['facts'][i], len(examples['facts'][i])))
        inputs.append('<|topic|> ' + topic + ' <|fact|> ' + fact +  ' <|style|> ' + style)

    model_inputs = tokenizer(inputs, max_length = max_input, padding = 'max_length', truncation = True)

    with tokenizer.as_target_tokenizer():
        targets = tokenizer(examples['output'], max_length = max_target, padding = 'max_length', truncation = True)

    model_inputs['labels'] = targets['input_ids']
    return model_inputs

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
tokenized_train_data = train_data.map(preprocess_function, batched = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

special_tokens_dict = {'additional_special_tokens': ['<|topic|>', '<|style|>', '<|fact|>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50268, 1024)

In [8]:
training_args = Seq2SeqTrainingArguments(
    output_dir = drive_path + '/paraphraser',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    weight_decay = 0.01,
    save_strategy = 'no',
    num_train_epochs = 4,
    gradient_accumulation_steps = 8
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_data,
    data_collator = data_collator,
    tokenizer = tokenizer
)

In [9]:
trainer.train()
trainer.save_model(drive_path + '/paraphraser/model')

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.602100


Step,Training Loss
500,1.602100
